In [1]:
import os
import cv2
import re
import numpy as np
from scipy import misc
from sklearn.svm import SVC
from sklearn import metrics
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix,greycoprops
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [7]:
#reading extracted features into X and labels into Y

#empty numpy float 64
#X=np.empty([833,5],dtype=np.float64)
#Y=np.empty([833,1],dtype=int)
X=[]
Y=[]

main_folder_path=r'C:\Users\Legion\Desktop\Internship_IIT\GRAZ'
segments=[r'bikes\bikes',r'persons\persons']

for seg in segments:
    absolute_path=os.path.join(main_folder_path,seg)
    
    for image_name in os.listdir(absolute_path):
        original__grey_image=misc.imread(os.path.join(absolute_path,image_name),mode='L')
        
        #inserting label 
        #bike's label is 0
        #person's label is 1
        z=re.match("^bike.",image_name)
        if z:
            Y.append(0)
        else:
            Y.append(1)
            
        
        #feature extraction using GLCM
        glcm_mat=greycomatrix(original__grey_image,[1],[0,np.pi/4,np.pi,0.75*np.pi],levels=256,symmetric=True,normed=True)
        #different features
        contrast=np.mean(greycoprops(glcm_mat,'contrast'))
        
        dissimilar=np.mean(greycoprops(glcm_mat,'dissimilarity'))
        
        homogeneity=np.mean(greycoprops(glcm_mat,'homogeneity'))
        
        energy=np.mean(greycoprops(glcm_mat,'energy'))
        
        asm=np.mean(greycoprops(glcm_mat,'ASM'))
        
        correlation=np.mean(greycoprops(glcm_mat,'correlation'))
        
        feature_vect=[correlation,dissimilar,homogeneity,energy,asm,correlation]
        
        X.append(feature_vect)
        
    

X=np.asarray(X,dtype=float)
Y=np.asarray(Y,dtype=int)
        

C:\Users\Legion\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()


In [8]:
print(Y.shape)

(833,)


In [9]:
#shuffling the feature vectors and labels together
randomise=np.arange(len(X))
np.random.shuffle(randomise)
X=X[randomise]
Y=Y[randomise]


#trainand test splitting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [10]:
#linear SVM
linearsvclassifier =SVC(kernel='linear')
linearsvclassifier.fit(X_train,Y_train)

Y_pred=linearsvclassifier.predict(X_test)

print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print("Accuracy : ",metrics.accuracy_score(Y_test,Y_pred))

[[35 37]
 [10 85]]
              precision    recall  f1-score   support

           0       0.78      0.49      0.60        72
           1       0.70      0.89      0.78        95

   micro avg       0.72      0.72      0.72       167
   macro avg       0.74      0.69      0.69       167
weighted avg       0.73      0.72      0.70       167

Accuracy :  0.718562874251497


In [11]:
#kernel SVM
kernelsvclassifier =SVC(kernel='rbf')
kernelsvclassifier.fit(X_train,Y_train)

Y_pred=kernelsvclassifier.predict(X_test)

print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print("Accuracy : ",metrics.accuracy_score(Y_test,Y_pred))

[[56 16]
 [44 51]]
              precision    recall  f1-score   support

           0       0.56      0.78      0.65        72
           1       0.76      0.54      0.63        95

   micro avg       0.64      0.64      0.64       167
   macro avg       0.66      0.66      0.64       167
weighted avg       0.67      0.64      0.64       167

Accuracy :  0.6407185628742516


C:\Users\Legion\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
